In [3]:
from abc import ABC, abstractmethod
import math

# Classe para gerenciar o tabuleiro
class Tabuleiro:
    def __init__(self):
        self.celulas = [' ' for _ in range(9)]

    def exibir(self):
        print(f"{self.celulas[0]} | {self.celulas[1]} | {self.celulas[2]}")
        print("--+---+--")
        print(f"{self.celulas[3]} | {self.celulas[4]} | {self.celulas[5]}")
        print("--+---+--")
        print(f"{self.celulas[6]} | {self.celulas[7]} | {self.celulas[8]}")

    def celula_vazia(self, indice):
        return self.celulas[indice] == ' '

    def fazer_jogada(self, indice, simbolo):
        if self.celula_vazia(indice):
            self.celulas[indice] = simbolo
            return True
        return False

    def verificar_vencedor(self, simbolo):
        combinacoes_vencedoras = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Linhas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Colunas
            [0, 4, 8], [2, 4, 6]              # Diagonais
        ]
        for combinacao in combinacoes_vencedoras:
            if all(self.celulas[i] == simbolo for i in combinacao):
                return True
        return False

    def tabuleiro_cheio(self):
        return ' ' not in self.celulas

    def limpar_tabuleiro(self):
        self.celulas = [' ' for _ in range(9)]

# Classe base para os jogadores
class Jogador(ABC):
    def __init__(self, nome, simbolo):
        self.nome = nome
        self.simbolo = simbolo

    @abstractmethod
    def fazer_jogada(self, tabuleiro):
        pass

# Classe para o jogador humano
class JogadorHumano(Jogador):
    def fazer_jogada(self, tabuleiro):
        print("\n--- Vez do jogador humano ---")
        while True:
            try:
                movimento = int(input(f"{self.nome}, escolha uma posição (0-8): "))
                if 0 <= movimento <= 8 and tabuleiro.celula_vazia(movimento):
                    return movimento
                
                print("Posição inválida ou ocupada. Tente novamente.")
            except ValueError:
                print("Entrada inválida. Digite um número entre 0 e 8.")

# Classe para a IA usando Minimax
class JogadorIA(Jogador):
    def fazer_jogada(self, tabuleiro):
        print(f"\n--- Vez do {self.nome} ({self.simbolo}) ---")
        movimento = self.minimax(tabuleiro, 0, True, -math.inf, math.inf)['posicao']
        return movimento

    def minimax(self, tabuleiro, profundidade, maximizando, alpha, beta):
        adversario = 'O' if self.simbolo == 'X' else 'X'
        
        if tabuleiro.verificar_vencedor(self.simbolo):
            return {'posicao': None, 'pontuacao': 10 - profundidade}
        if tabuleiro.verificar_vencedor(adversario):
            return {'posicao': None, 'pontuacao': profundidade - 10}
        if tabuleiro.tabuleiro_cheio():
            return {'posicao': None, 'pontuacao': 0}

        melhor_jogada = {'posicao': None, 'pontuacao': -math.inf if maximizando else math.inf}
        
        for i in range(9):
            if tabuleiro.celula_vazia(i):
                tabuleiro.fazer_jogada(i, self.simbolo if maximizando else adversario)
                sim_pontuacao = self.minimax(tabuleiro, profundidade + 1, not maximizando, alpha, beta)
                tabuleiro.celulas[i] = ' '
                sim_pontuacao['posicao'] = i
                
                if maximizando:
                    if sim_pontuacao['pontuacao'] > melhor_jogada['pontuacao']:
                        melhor_jogada = sim_pontuacao
                    alpha = max(alpha, sim_pontuacao['pontuacao'])
                else:
                    if sim_pontuacao['pontuacao'] < melhor_jogada['pontuacao']:
                        melhor_jogada = sim_pontuacao
                    beta = min(beta, sim_pontuacao['pontuacao'])
                
                if beta <= alpha:
                    break  # Poda alfa-beta
        
        return melhor_jogada

    # def minimax(self, tabuleiro, jogador_atual):
        # Verifica se o jogo terminou
        if tabuleiro.verificar_vencedor(self.simbolo):
            return {'posicao': None, 'pontuacao': 1}
        elif tabuleiro.verificar_vencedor('O' if self.simbolo == 'X' else 'X'):
            return {'posicao': None, 'pontuacao': -1}
        elif tabuleiro.tabuleiro_cheio():
            return {'posicao': None, 'pontuacao': 0}

        # Inicializa o dicionário de melhor jogada
        melhor_jogada = {'posicao': None, 'pontuacao': -math.inf if jogador_atual == self.simbolo else math.inf}

        for i in range(9):
            if tabuleiro.celula_vazia(i):
                tabuleiro.fazer_jogada(i, jogador_atual)
                sim_pontuacao = self.minimax(tabuleiro, 'O' if jogador_atual == 'X' else 'X')

                # Desfaz a jogada
                tabuleiro.celulas[i] = ' '
                sim_pontuacao['posicao'] = i

                # Atualiza a melhor jogada
                if jogador_atual == self.simbolo:
                    if sim_pontuacao['pontuacao'] > melhor_jogada['pontuacao']:
                        melhor_jogada = sim_pontuacao
                else:
                    if sim_pontuacao['pontuacao'] < melhor_jogada['pontuacao']:
                        melhor_jogada = sim_pontuacao

        return melhor_jogada

# Classe principal do jogo
class JogoDaVelha:
    def __init__(self):
        self.tabuleiro = Tabuleiro()
        self.jogador_x = None
        self.jogador_o = None

    def escolher_simbolo(self):
        while True:
            escolha = input("Escolha seu símbolo (X ou O): ").upper()
            if escolha in ['X', 'O']:
                return escolha
            else:
                print("Escolha inválida. Digite 'X' ou 'O'.")

    def configurar_jogadores(self):
        nome_humano = input("Digite seu nome: ")
        simbolo_humano = self.escolher_simbolo()
        simbolo_ia = 'O' if simbolo_humano == 'X' else 'X'

        self.jogador_x = JogadorHumano(nome_humano, simbolo_humano) if simbolo_humano == 'X' else JogadorIA("Computador", simbolo_ia)
        self.jogador_o = JogadorIA("Computador", simbolo_ia) if simbolo_humano == 'X' else JogadorHumano(nome_humano, simbolo_humano)

    def jogar(self):
        self.tabuleiro.limpar_tabuleiro()
        self.tabuleiro.exibir()
        self.configurar_jogadores()

        jogador_atual = self.jogador_x  # X sempre começa

        while True:
            # Faz a jogada
            movimento = jogador_atual.fazer_jogada(self.tabuleiro)
            self.tabuleiro.fazer_jogada(movimento, jogador_atual.simbolo)
            self.tabuleiro.exibir()

            # Verifica se há um vencedor
            if self.tabuleiro.verificar_vencedor(jogador_atual.simbolo):
                print(f"Parabéns, {jogador_atual.nome} venceu!")
                break
            if self.tabuleiro.tabuleiro_cheio():
                print("O jogo terminou em empate!")
                break

            # Alterna para o próximo jogador
            jogador_atual = self.jogador_o if jogador_atual == self.jogador_x else self.jogador_x


In [6]:
# Iniciar o jogo
jogo = JogoDaVelha()
jogo.jogar()

  |   |  
--+---+--
  |   |  
--+---+--
  |   |  

--- Vez do jogador humano ---
X |   |  
--+---+--
  |   |  
--+---+--
  |   |  

--- Vez do Computador (O) ---
X |   |  
--+---+--
  | O |  
--+---+--
  |   |  

--- Vez do jogador humano ---
X |   | X
--+---+--
  | O |  
--+---+--
  |   |  

--- Vez do Computador (O) ---
X | O | X
--+---+--
  | O |  
--+---+--
  |   |  

--- Vez do jogador humano ---
X | O | X
--+---+--
  | O |  
--+---+--
  | X |  

--- Vez do Computador (O) ---
X | O | X
--+---+--
O | O |  
--+---+--
  | X |  

--- Vez do jogador humano ---
X | O | X
--+---+--
O | O | X
--+---+--
  | X |  

--- Vez do Computador (O) ---
X | O | X
--+---+--
O | O | X
--+---+--
  | X | O

--- Vez do jogador humano ---
X | O | X
--+---+--
O | O | X
--+---+--
X | X | O
O jogo terminou em empate!
